In [1]:
from eppy import modeleditor
from eppy.modeleditor import IDF
from matplotlib import pyplot as plt
import os
import pandas as pd
from besos import eppy_funcs as ef
from besos import sampling
from besos.evaluator import EvaluatorEP
from besos.parameters import RangeParameter, FieldSelector, FilterSelector, Parameter, expand_plist, wwr, CategoryParameter, GenericSelector
from besos.problem import EPProblem

Define the IDF, IDD and the weather file

In [2]:
idd_file='/usr/local/EnergyPlus-9-0-1/Energy+.idd'
idf_3 = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/3_layer_window_opt.idf"
weather_file = '/home/ict4bd/Desktop/ict4building/Requirements/CHE_Geneva.067000_IWEC/custom_epw_file.epw'
weather_2002 = '/home/ict4bd/Desktop/ict4building/Requirements/CHE_Geneva.067000_IWEC/CHE_Geneva.067000_IWEC.epw'

In [3]:
building = ef.get_building(idf_3)

In [4]:
# Parameter Space
parameters = []

orientation = FieldSelector(class_name='Building', field_name='North Axis')
orientation_descriptor = CategoryParameter(options=[0, 90, 180, 270])

insulation = FieldSelector(class_name='Material', object_name='XPS Extruded Polystyrene  - CO2 Blowing_.4359', 
                           field_name='Thickness')
insulation_descriptor = RangeParameter(min_val = 0.01, max_val=0.5) 

shading = FieldSelector(class_name='WindowShadingControl',field_name='Setpoint')
shading_descriptor = RangeParameter(min_val = 0, max_val=30) 

ventilation = FieldSelector(class_name='ZoneVentilation:DesignFlowRate',field_name='Design Flow Rate')
ventilation_descriptor = CategoryParameter(options=[0, 0.12])

# parameters = [Orientation, ventilation, shading, insulation, wwr]
parameters.append(Parameter(selector=orientation, 
                            value_descriptor=orientation_descriptor,
                            name='Orientation'))

parameters.append(Parameter(selector=ventilation,
                            value_descriptor=ventilation_descriptor,
                            name='Ventilation'))

parameters.append(Parameter(selector=shading,
                            value_descriptor=shading_descriptor,
                            name='Shading'))

parameters.append(Parameter(selector=insulation,
                            value_descriptor=insulation_descriptor,
                            name='Insulation Thickness'))

parameters.append(wwr(RangeParameter(0.1,0.9))) 

# Objectives of our problem which is to minimize the TOTAL COMSUMPTION
objectives = ['Electricity:Facility','DistrictCooling:Facility', 'DistrictHeating:Facility']

# problem = parameters + objectives
besos_problem = EPProblem(parameters, objectives)

# EPProblem vs EvaluatorEP = problem + building + weather
evaluator = EvaluatorEP(besos_problem, building, out_dir='besos_opt', err_dir='Silent', epw_file=weather_file)

/home/ict4bd/py3/lib/python3.8/site-packages/besos/evaluator.py:659: FutureWarning: epw_file has been deprecated and will be removed in the future. Use epw instead.
  warn(


In [5]:
samples_temp =[]
samples_temp.append({'Orientation': 90,
                     'Ventilation': 0,
                     'Shading': 9.514906,
                     'Insulation Thickness': 0.435923,
                     'Window to Wall Ratio': 0.105726,})
samples = pd.DataFrame.from_dict(samples_temp)
samples

,Orientation,Ventilation,Shading,Insulation Thickness,Window to Wall Ratio
0,90,0,9.514906,0.435923,0.105726


In [6]:
outputs = evaluator.df_apply(samples, keep_input=True)    

In [7]:
IDF.setiddname(idd_file)
#idf = IDF(idf_3,weather_file)
idf = IDF(idf_3,weather_2002)

In [8]:
idf.run(readvars=True,output_directory=u'eppy_opt',annual= True)


/usr/local/EnergyPlus-9-0-1/energyplus --weather /home/ict4bd/Desktop/ict4building/Requirements/CHE_Geneva.067000_IWEC/CHE_Geneva.067000_IWEC.epw --output-directory /home/ict4bd/Desktop/ict4building/Simulation/eppy_opt --annual --idd /usr/local/EnergyPlus-9-0-1/Energy+.idd --readvars /home/ict4bd/Desktop/ict4building/Simulation/in.idf



In [9]:
csv_columns = ['Date/Time',
              'BLOCK1:ZONE3:Zone Operative Temperature [C](Hourly)',
              'BLOCK1:ZONE1:Zone Operative Temperature [C](Hourly)',
              'BLOCK1:ZONE2:Zone Operative Temperature [C](Hourly)',
              'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)',
              'DistrictHeating:Facility [J](Hourly)',
              'DistrictCooling:Facility [J](Hourly)',
              'Electricity:Facility [J](Hourly)']

In [10]:
df_columns = {'Date/Time':'Date_Time',
              'BLOCK1:ZONE3:Zone Operative Temperature [C](Hourly)':'t_in_Zone3',
              'BLOCK1:ZONE1:Zone Operative Temperature [C](Hourly)':'t_in_Zone1',
              'BLOCK1:ZONE2:Zone Operative Temperature [C](Hourly)':'t_in_Zone2',
              'Environment:Site Outdoor Air Drybulb Temperature [C](Hourly)':'t_out',
              'DistrictHeating:Facility [J](Hourly)':'power_heating',
              'DistrictCooling:Facility [J](Hourly)':'power_cooling' ,
              'Electricity:Facility [J](Hourly)':'power_electricity'}

In [11]:
idf_data =pd.read_csv('eppy_opt/eplusout.csv',usecols=csv_columns)

In [12]:
idf_data = idf_data.rename(columns =df_columns)

In [13]:
idf_data['t_in'] = idf_data[['t_in_Zone3','t_in_Zone1','t_in_Zone2']].mean(axis=1)
idf_data.drop(['t_in_Zone3','t_in_Zone1','t_in_Zone2'], axis = 1)
idf_data['delta_t'] =idf_data['t_in'] - idf_data['t_out']

In [14]:
idf_data.dropna(inplace= True)
idf_data

,Date_Time,t_out,t_in_Zone3,t_in_Zone1,t_in_Zone2,power_electricity,power_cooling,power_heating,t_in,delta_t
1,01/01 01:00:00,-1.975,19.807956,19.824910,19.771249,0.0,0.0,3.132761e+06,19.801372,21.776372
3,01/01 02:00:00,-1.950,19.804777,19.820755,19.767485,0.0,0.0,3.141718e+06,19.797672,21.747672
5,01/01 03:00:00,-1.500,19.802587,19.818087,19.765032,0.0,0.0,3.087815e+06,19.795236,21.295236
7,01/01 04:00:00,-1.400,19.800607,19.815862,19.762766,0.0,0.0,3.079862e+06,19.793078,21.193078
9,01/01 05:00:00,-1.475,19.798827,19.813976,19.760709,0.0,0.0,3.095619e+06,19.791171,21.266171
...,...,...,...,...,...,...,...,...,...,...
17511,12/31 20:00:00,-3.475,19.810530,19.839042,19.807078,1266457.5,0.0,4.144298e+06,19.818883,23.293883
17513,12/31 21:00:00,-3.825,19.812981,19.840159,19.806422,1266457.5,0.0,4.188553e+06,19.819854,23.644854
17515,12/31 22:00:00,-3.975,19.815155,19.841129,19.806040,1266457.5,0.0,3.396859e+06,19.820775,23.795775
17517,12/31 23:00:00,-3.850,19.815401,19.840228,19.803903,361845.0,0.0,3.198604e+06,19.819844,23.669844


In [15]:
# idf_data['Date_Time'] = '2017/' + idf_data['Date_Time'].str.strip()
idf_data['Date_Time'] = '2002/' + idf_data['Date_Time'].str.strip()
idf_data['Date_Time'] = idf_data['Date_Time'].str.replace('24:00:00','23:59:59')
idf_data

,Date_Time,t_out,t_in_Zone3,t_in_Zone1,t_in_Zone2,power_electricity,power_cooling,power_heating,t_in,delta_t
1,2002/01/01 01:00:00,-1.975,19.807956,19.824910,19.771249,0.0,0.0,3.132761e+06,19.801372,21.776372
3,2002/01/01 02:00:00,-1.950,19.804777,19.820755,19.767485,0.0,0.0,3.141718e+06,19.797672,21.747672
5,2002/01/01 03:00:00,-1.500,19.802587,19.818087,19.765032,0.0,0.0,3.087815e+06,19.795236,21.295236
7,2002/01/01 04:00:00,-1.400,19.800607,19.815862,19.762766,0.0,0.0,3.079862e+06,19.793078,21.193078
9,2002/01/01 05:00:00,-1.475,19.798827,19.813976,19.760709,0.0,0.0,3.095619e+06,19.791171,21.266171
...,...,...,...,...,...,...,...,...,...,...
17511,2002/12/31 20:00:00,-3.475,19.810530,19.839042,19.807078,1266457.5,0.0,4.144298e+06,19.818883,23.293883
17513,2002/12/31 21:00:00,-3.825,19.812981,19.840159,19.806422,1266457.5,0.0,4.188553e+06,19.819854,23.644854
17515,2002/12/31 22:00:00,-3.975,19.815155,19.841129,19.806040,1266457.5,0.0,3.396859e+06,19.820775,23.795775
17517,2002/12/31 23:00:00,-3.850,19.815401,19.840228,19.803903,361845.0,0.0,3.198604e+06,19.819844,23.669844


In [16]:
idf_data['power_heating'] /= 3.6e6
idf_data['power_cooling'] /= 3.6e6
idf_data['power_electricity'] /= 3.6e6

In [17]:
idf_data['power_heating'] = idf_data['power_heating'].apply(lambda x: round(x, 2))
idf_data['power_cooling'] = idf_data['power_cooling'].apply(lambda x: round(x, 2))
idf_data['power_electricity'] = idf_data['power_electricity'].apply(lambda x: round(x, 2))
idf_data['t_in'] = idf_data['t_in'].apply(lambda x: round(x, 2))
idf_data['t_out'] = idf_data['t_out'].apply(lambda x: round(x, 2))

In [18]:
idf_data['total_power'] = idf_data['power_heating']+ idf_data['power_cooling'] + idf_data['power_electricity']

In [19]:
idf_data = idf_data[['Date_Time','t_in','t_out','delta_t','power_heating','power_cooling','power_electricity','total_power']]

In [20]:
idf_data.to_csv('simulation_1Year_2002.csv',index=False)

In [21]:
idf_check =pd.read_csv('simulation_1Year_2002.csv')
idf_check
# kWh total power 
# total area is 41 m^2
# if we want Q then divide

,Date_Time,t_in,t_out,delta_t,power_heating,power_cooling,power_electricity,total_power
0,2002/01/01 01:00:00,19.80,-1.98,21.776372,0.87,0.0,0.00,0.87
1,2002/01/01 02:00:00,19.80,-1.95,21.747672,0.87,0.0,0.00,0.87
2,2002/01/01 03:00:00,19.80,-1.50,21.295236,0.86,0.0,0.00,0.86
3,2002/01/01 04:00:00,19.79,-1.40,21.193078,0.86,0.0,0.00,0.86
4,2002/01/01 05:00:00,19.79,-1.48,21.266171,0.86,0.0,0.00,0.86
...,...,...,...,...,...,...,...,...
8755,2002/12/31 20:00:00,19.82,-3.48,23.293883,1.15,0.0,0.35,1.50
8756,2002/12/31 21:00:00,19.82,-3.83,23.644854,1.16,0.0,0.35,1.51
8757,2002/12/31 22:00:00,19.82,-3.98,23.795775,0.94,0.0,0.35,1.29
8758,2002/12/31 23:00:00,19.82,-3.85,23.669844,0.89,0.0,0.10,0.99
